In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline


In [7]:
df = pd.read_csv('../data/data_171_cols.csv').drop(['Unnamed: 0'], 1)
df

<ipython-input-7-51ba623ac62c>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = pd.read_csv('../data/data_171_cols.csv').drop(['Unnamed: 0'], 1)


,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,championPoints100_2,championPoints100_3,championPoints100_4,championPoints100_5,championPoints200_6,championPoints200_7,championPoints200_8,championPoints200_9,championPoints200_10,winner
0,0,2741,3045,3545,2789,2026,4380,4114,3372,3419,...,37664.0,40303.0,96304.0,608721.0,1180052.0,45215.0,118831.0,16631.0,89581.0,200
1,1,2888,4549,3947,3600,2678,3715,2891,3003,3084,...,890336.0,605169.0,151187.0,548849.0,404369.0,423591.0,16728.0,14532.0,12363.0,200
2,10,2664,3850,3712,4927,2841,4655,3186,4058,2968,...,190520.0,16424.0,67522.0,5179.0,81384.0,15555.0,171780.0,38171.0,76948.0,100
3,100,2814,4072,3289,5535,3880,3516,4619,2940,2653,...,64717.0,2703244.0,220056.0,639229.0,1676567.0,236945.0,277.0,185622.0,552615.0,100
4,1000,3164,4548,3811,3998,3808,4078,3602,3068,2830,...,99843.0,20610.0,13271.0,89289.0,62110.0,88791.0,7079.0,385086.0,26974.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,995,3368,3486,3951,3682,2280,3232,3053,3835,3174,...,6910.0,56731.0,17585.0,24890.0,20145.0,702664.0,31861.0,28809.0,34855.0,100
7996,996,3842,3779,3663,4833,2739,3209,3274,3461,3667,...,2700.0,373159.0,33958.0,20172.0,1172.0,11735.0,2689100.0,256726.0,50281.0,100
7997,997,3583,3761,3037,3093,2325,4141,3821,3343,4427,...,82340.0,8594.0,33273.0,387.0,377117.0,4647.0,12031.0,99127.0,130341.0,200
7998,998,3000,3458,2832,3846,2462,4427,4389,3161,2770,...,2662.0,103213.0,6158.0,205005.0,2095625.0,60531.0,776567.0,25860.0,20679.0,200


# MERGING patch data

In [12]:
patchByLaneTrain = pd.read_csv('../data/patchByLaneTrain.csv')
patchByLaneTrain

,matchId,champWinPercent100_1,champWinPercent100_2,champWinPercent100_3,champWinPercent100_4,champWinPercent100_5,champWinPercent200_6,champWinPercent200_7,champWinPercent200_8,champWinPercent200_9,...,champScore100_1,champScore100_2,champScore100_3,champScore100_4,champScore100_5,champScore200_6,champScore200_7,champScore200_8,champScore200_9,champScore200_10
0,0,0.5193,0.4721,0.5193,0.4970,0.5131,0.5187,0.5138,0.5252,0.5050,...,62.28,45.14,49.87,84.23,54.52,43.31,94.80,60.29,47.66,47.55
1,1,0.5124,0.5080,0.4903,0.5227,0.5108,0.5173,0.5237,0.5117,0.5222,...,76.26,58.21,47.55,68.02,96.39,76.08,64.41,63.02,68.26,59.02
2,2,0.4849,0.5408,0.5328,0.5163,0.5107,0.5060,0.5190,0.5058,0.4866,...,49.47,55.12,94.56,93.98,52.08,89.52,53.91,53.65,45.74,48.09
3,3,0.4894,0.5217,0.5056,0.4970,0.4858,0.5173,0.4893,0.5109,0.5333,...,55.02,59.11,59.91,84.23,43.82,76.08,53.95,74.42,79.18,54.17
4,4,0.4801,0.5281,0.5395,0.5140,0.4954,0.4728,0.4695,0.5035,0.5168,...,59.11,57.15,49.15,59.57,48.82,52.02,41.17,65.25,44.49,68.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,0.5109,0.5462,0.4956,0.5210,0.5189,0.5298,0.5358,0.4762,0.5167,...,74.42,72.85,55.35,37.49,54.89,45.08,46.48,43.80,43.11,42.00
7996,7996,0.5387,0.5281,0.4907,0.4805,0.5249,0.4833,0.5138,0.5237,0.5140,...,44.79,57.15,46.57,70.70,70.96,44.16,94.80,41.66,59.57,48.11
7997,7997,0.5401,0.5241,0.4985,0.5222,0.5144,0.5300,0.4837,0.4565,0.5227,...,50.62,52.68,47.30,68.26,68.98,67.60,61.51,44.09,68.02,70.96
7998,7998,0.5471,0.5281,0.5204,0.5333,0.5249,0.4995,0.4839,0.5149,0.5050,...,62.92,57.15,47.33,79.18,70.96,56.47,37.60,52.22,47.66,39.18


In [13]:
df_p = pd.merge(df, patchByLaneTrain, how = 'inner', on = 'matchId')
df_p

,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,champScore100_1,champScore100_2,champScore100_3,champScore100_4,champScore100_5,champScore200_6,champScore200_7,champScore200_8,champScore200_9,champScore200_10
0,0,2741,3045,3545,2789,2026,4380,4114,3372,3419,...,62.28,45.14,49.87,84.23,54.52,43.31,94.80,60.29,47.66,47.55
1,1,2888,4549,3947,3600,2678,3715,2891,3003,3084,...,76.26,58.21,47.55,68.02,96.39,76.08,64.41,63.02,68.26,59.02
2,10,2664,3850,3712,4927,2841,4655,3186,4058,2968,...,44.08,42.27,91.12,68.02,48.50,68.26,72.47,55.35,90.92,54.52
3,100,2814,4072,3289,5535,3880,3516,4619,2940,2653,...,52.06,72.47,54.47,63.75,70.96,50.62,55.34,49.15,43.11,57.01
4,1000,3164,4548,3811,3998,3808,4078,3602,3068,2830,...,61.67,64.82,59.06,61.42,52.83,73.48,70.20,59.08,74.42,96.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,995,3368,3486,3951,3682,2280,3232,3053,3835,3174,...,62.28,72.85,54.23,59.57,70.96,44.16,47.06,60.29,84.23,49.40
7996,996,3842,3779,3663,4833,2739,3209,3274,3461,3667,...,49.00,41.74,47.81,51.26,67.81,60.41,37.75,58.02,74.02,44.87
7997,997,3583,3761,3037,3093,2325,4141,3821,3343,4427,...,44.08,72.47,91.12,79.18,40.05,76.26,46.43,48.86,43.84,43.82
7998,998,3000,3458,2832,3846,2462,4427,4389,3161,2770,...,60.06,34.39,56.66,93.98,52.08,70.04,62.35,53.65,46.79,67.81


In [15]:
#columns with nans
df_p.columns[df_p.isna().any()].tolist()

['championLevel100_1',
 'championLevel100_3',
 'championLevel100_4',
 'championLevel100_5',
 'championLevel200_6',
 'championLevel200_8',
 'championLevel200_10',
 'championPoints100_1',
 'championPoints100_3',
 'championPoints100_4',
 'championPoints100_5',
 'championPoints200_6',
 'championPoints200_8',
 'championPoints200_10']

In [16]:
#trying with nan rows dropped -- gives higher accuracy
df_p=df_p.dropna()

In [17]:
df_p.isna().sum()

matchId             0
totalGold100_1      0
totalGold100_2      0
totalGold100_3      0
totalGold100_4      0
                   ..
champScore200_6     0
champScore200_7     0
champScore200_8     0
champScore200_9     0
champScore200_10    0
Length: 192, dtype: int64

In [18]:
#trying with nan columns dropped
df3 = df.drop(['championLevel100_4',
 'championLevel200_7',
 'championPoints100_4',
 'championPoints200_7', 'championLevel100_1',
 'championLevel100_3',

 'championLevel100_5',
 'championLevel200_6',
 'championLevel200_8',
 'championLevel200_10',
 'championPoints100_1',
 'championPoints100_3',
 
 'championPoints100_5',
 'championPoints200_6',
 'championPoints200_8',
 'championPoints200_10'], axis = 1)

from https://www.analyticsvidhya.com/blog/2021/09/gradient-boosting-algorithm-a-complete-guide-for-beginners/

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [20]:
#categorical columns need special treatment:
categoricals = [col for col in df_p if col.startswith('championName')]
categoricals

['championName100_1',
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10']

In [ ]:
#X[categoricals] = X[categoricals].astype('category')

In [21]:
X = df_p.drop(['winner',  'championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [22]:
y = df_p['winner']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [24]:
X_train.shape

(6390, 181)

In [25]:
gbc=GradientBoostingClassifier(n_estimators=500,learning_rate=0.05,random_state=100,max_features=5 )

In [26]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [27]:
print(confusion_matrix(y_test, gbc.predict(X_test)))

[[615 206]
 [235 542]]


In [28]:
print("GBC accuracy is %2.4f" % accuracy_score( 
     y_test, gbc.predict(X_test)))

GBC accuracy is 0.7240


'gbc' gives 72.4

In [29]:
from sklearn.metrics import classification_report

pred=gbc.predict(X_test)

print(classification_report(y_test, pred))


              precision    recall  f1-score   support

         100       0.72      0.75      0.74       821
         200       0.72      0.70      0.71       777

    accuracy                           0.72      1598
   macro avg       0.72      0.72      0.72      1598
weighted avg       0.72      0.72      0.72      1598



**Tuning

In [30]:
from sklearn.model_selection import GridSearchCV

grid = {

    'learning_rate':[0.01,0.05,0.1],

    'n_estimators':np.arange(100,500,100),

}


gb = GradientBoostingClassifier()

gb_cv = GridSearchCV(gb, grid, cv = 4)

gb_cv.fit(X_train,y_train)

print("Best Parameters:",gb_cv.best_params_)

print("Train Score:",gb_cv.best_score_)

print("Test Score:",gb_cv.score(X_test,y_test))

Best Parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Train Score: 0.7178416704349441
Test Score: 0.7215269086357947


gb_cv gives 72.15

**Tuning max_depth**

gb_cv_md - use gb_cv (72.15) and tune for max depth 

In [31]:
grid = {'max_depth':[2,3,4,5,6,7] }

gb = GradientBoostingClassifier(learning_rate=0.1,n_estimators=100)

gb_cv_md = GridSearchCV(gb, grid, cv = 4)

gb_cv_md.fit(X_train,y_train)

print("Best Parameters:",gb_cv_md.best_params_)

print("Train Score:",gb_cv_md.best_score_)

print("Test Score:",gb_cv_md.score(X_test,y_test))

Best Parameters: {'max_depth': 3}
Train Score: 0.7183110071057827
Test Score: 0.72090112640801


gbc_cv_md --tuned for max depth gives 72.09
**--using gb_cv (tuned for learning rate, n_estimators)**

# Predict on Test data

In [32]:
folder_path = '../data/test_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [33]:
participantid = []
currentgold = []
totalgold = []
minionskilled = []
xp = []
matchid = []
teamid = []
cs_abilityHaste = []
cs_abilityPower = []
cs_armor = []
cs_attackDamage = []
cs_attackSpeed = []
cs_health = []
cs_healthMax = []
dd_totalDamageTaken = []
dd_pDamageDoneToChamps = []
dd_totDamageDone = []
positionX = []
positionY = []



def file_buster(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        minionskilled.append(j['minionsKilled'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')
        cs_stats = j['championStats']
        cs_abilityHaste.append(cs_stats['abilityHaste'])
        cs_abilityPower.append(cs_stats['abilityPower'])
        cs_armor.append(cs_stats['armor'])
        cs_attackDamage.append(cs_stats['attackDamage'])
        cs_attackSpeed.append(cs_stats['attackSpeed'])
        cs_health.append(cs_stats['health'])
        cs_healthMax.append(cs_stats['healthMax'])
    
        d_stats = j['damageStats']
        dd_totalDamageTaken.append(d_stats['totalDamageTaken'])
        dd_pDamageDoneToChamps.append(d_stats['totalDamageDoneToChampions'])
        dd_totDamageDone.append(d_stats['totalDamageDone'])
    
        pos = j['position']
        positionX.append(pos['x'])
        positionY.append(pos['y']) 


In [34]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    file_buster(file)
    count+=1
print(str(count) + " files loaded")

2000 files loaded


In [35]:
di = {'matchId':matchid, 'teamId':teamid, 'participantId' : participantid, 'currentGold': currentgold, \
      'totalGold':totalgold, 'xp': xp, 'cs_abilityHaste': cs_abilityHaste, 'cs_abilityPower':cs_abilityPower,\
     'cs_armor': cs_armor, 'cs_attackDamage': cs_attackDamage,'cs_attackSpeed':cs_attackSpeed, \
     'cs_health': cs_health, 'cs_healthMax': cs_healthMax, 'dd_totalDamageTaken':dd_totalDamageTaken, \
     'dd_pDamageDoneToChamps' : dd_pDamageDoneToChamps, 'dd_totDamageDone': dd_totDamageDone, \
     'positionX': positionX, 'positionY': positionY}
za = pd.DataFrame(di)
za    

,matchId,teamId,participantId,currentGold,totalGold,xp,cs_abilityHaste,cs_abilityPower,cs_armor,cs_attackDamage,cs_attackSpeed,cs_health,cs_healthMax,dd_totalDamageTaken,dd_pDamageDoneToChamps,dd_totDamageDone,positionX,positionY
0,8000,100,1,1513,3063,4659,0,0,66,110,128,774,1342,3513,1159,21426,3127,12922
1,8000,100,2,1742,4117,3539,0,0,65,141,138,535,1153,8469,3660,45005,7949,8549
2,8000,100,3,1015,4365,4431,0,191,57,67,112,1451,1518,1949,2166,29146,8052,8160
3,8000,100,4,80,3555,3061,0,0,49,131,183,1077,1077,2352,3032,17927,11236,1751
4,8000,100,5,78,2213,2409,0,46,51,61,108,1081,1214,2633,1062,2604,11482,1563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,217,2852,4621,0,69,67,84,115,1052,1475,5129,3780,19606,3633,12989
19996,9999,200,7,1510,2860,2905,0,58,58,65,108,885,1068,8419,1111,48297,7778,10713
19997,9999,200,8,495,3245,4214,0,0,50,113,245,295,1339,5560,3940,22466,6292,7209
19998,9999,200,9,283,2483,2565,0,0,44,108,113,952,1106,1824,903,15240,13859,4119


In [36]:
champion_mastery = pd.read_csv('../data/champion_mastery.csv')
participants_test = pd.read_csv('../data/participants_test.csv')

In [37]:
champion_points = pd.merge(participants_test, champion_mastery, on = ['summonerId', 'championId'], how = 'left' ).drop(['chestGranted', 'tokensEarned'], axis = 1)
champion_points

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints
0,8000,100,1,23938,57,Fiora,114,3.0,10896.0
1,8000,100,2,8037,788,LeeSin,64,7.0,143994.0
2,8000,100,3,8444,566,Veigar,45,7.0,203012.0
3,8000,100,4,24831,542,Twitch,29,7.0,376388.0
4,8000,100,5,23890,224,Soraka,16,7.0,625009.0
...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,45320,458,Mordekaiser,82,7.0,117779.0
19996,9999,200,7,45321,330,FiddleSticks,9,5.0,22922.0
19997,9999,200,8,45322,462,Tristana,18,7.0,169934.0
19998,9999,200,9,45323,256,Samira,360,4.0,19716.0


In [38]:
#convert to int types
coltomergeon = ['matchId', 'teamId', 'participantId']

for x in za[coltomergeon]:
    za[x] = za[x].astype(int)
    
for x in champion_points[coltomergeon]:
    champion_points[x] = champion_points[x].astype(int)

In [39]:
test_data = pd.merge(za, champion_points, on = ['matchId', 'teamId', 'participantId'], how = 'left')
test_data

,matchId,teamId,participantId,currentGold,totalGold,xp,cs_abilityHaste,cs_abilityPower,cs_armor,cs_attackDamage,...,dd_pDamageDoneToChamps,dd_totDamageDone,positionX,positionY,summonerId,summonerLevel,championName,championId,championLevel,championPoints
0,8000,100,1,1513,3063,4659,0,0,66,110,...,1159,21426,3127,12922,23938,57,Fiora,114,3.0,10896.0
1,8000,100,2,1742,4117,3539,0,0,65,141,...,3660,45005,7949,8549,8037,788,LeeSin,64,7.0,143994.0
2,8000,100,3,1015,4365,4431,0,191,57,67,...,2166,29146,8052,8160,8444,566,Veigar,45,7.0,203012.0
3,8000,100,4,80,3555,3061,0,0,49,131,...,3032,17927,11236,1751,24831,542,Twitch,29,7.0,376388.0
4,8000,100,5,78,2213,2409,0,46,51,61,...,1062,2604,11482,1563,23890,224,Soraka,16,7.0,625009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,217,2852,4621,0,69,67,84,...,3780,19606,3633,12989,45320,458,Mordekaiser,82,7.0,117779.0
19996,9999,200,7,1510,2860,2905,0,58,58,65,...,1111,48297,7778,10713,45321,330,FiddleSticks,9,5.0,22922.0
19997,9999,200,8,495,3245,4214,0,0,50,113,...,3940,22466,6292,7209,45322,462,Tristana,18,7.0,169934.0
19998,9999,200,9,283,2483,2565,0,0,44,108,...,903,15240,13859,4119,45323,256,Samira,360,4.0,19716.0


In [40]:
results = pd.DataFrame({'matchId': test_data['matchId'].unique()})

cols_to_pivot = ['totalGold', 'xp',  'cs_abilityHaste', 'cs_abilityPower','cs_armor', 'cs_attackDamage','cs_attackSpeed', \
     'cs_health', 'cs_healthMax', 'dd_totalDamageTaken', \
     'dd_pDamageDoneToChamps' ,'dd_totDamageDone', \
     'positionX', 'positionY' , 'championName', 'championLevel', 'championPoints' ]


for i in test_data[cols_to_pivot]:
    print(i)
    pivot_table = pd.pivot(test_data, values = i, index = 'matchId', columns = ['teamId', 'participantId'])
    
    pivot_table.columns = [str(i)+str('_'.join(map(str, c))) for c in pivot_table]
    
    results = pd.merge(results, pivot_table, on='matchId', how = 'inner')
    print(results.shape)
    
   

totalGold
(2000, 11)
xp
(2000, 21)
cs_abilityHaste
(2000, 31)
cs_abilityPower
(2000, 41)
cs_armor
(2000, 51)
cs_attackDamage
(2000, 61)
cs_attackSpeed
(2000, 71)
cs_health
(2000, 81)
cs_healthMax
(2000, 91)
dd_totalDamageTaken
(2000, 101)
dd_pDamageDoneToChamps
(2000, 111)
dd_totDamageDone
(2000, 121)
positionX
(2000, 131)
positionY
(2000, 141)
championName
(2000, 151)
championLevel
(2000, 161)
championPoints
(2000, 171)


In [41]:
results.shape

(2000, 171)

In [42]:
results

,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,championPoints100_1,championPoints100_2,championPoints100_3,championPoints100_4,championPoints100_5,championPoints200_6,championPoints200_7,championPoints200_8,championPoints200_9,championPoints200_10
0,8000,3063,4117,4365,3555,2213,3075,3884,3024,4150,...,10896.0,143994.0,203012.0,376388.0,625009.0,5409.0,65370.0,221232.0,280935.0,375328.0
1,8001,3784,3548,3203,4193,2764,3048,4045,2582,4168,...,474445.0,331803.0,36367.0,58213.0,246425.0,837077.0,299153.0,47659.0,4328.0,31819.0
2,8002,2681,4605,4309,2569,2042,3094,2827,3971,3504,...,164164.0,7589.0,102137.0,46963.0,57280.0,16372.0,24916.0,74657.0,53350.0,8113.0
3,8003,3471,3808,4186,4028,2291,4803,4072,2681,3130,...,49639.0,399780.0,12132.0,243849.0,74381.0,12078.0,16401.0,35506.0,10298.0,29322.0
4,8004,3181,3569,3693,3274,2624,3210,3997,3127,4044,...,3549.0,35560.0,142868.0,46016.0,894152.0,136693.0,179980.0,42794.0,14187.0,18014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,3735,2545,4207,2930,2023,2992,3100,3282,3288,...,138017.0,66051.0,58901.0,134002.0,237322.0,44090.0,5654.0,124155.0,32877.0,21563.0
1996,9996,4162,4277,3313,4289,3082,3253,3173,4187,4079,...,252137.0,53232.0,827163.0,10702.0,54286.0,696929.0,12893.0,108602.0,80170.0,717229.0
1997,9997,2747,3392,2755,3269,2556,3728,3391,3492,4135,...,14514.0,16975.0,226507.0,106182.0,17522.0,95755.0,1856187.0,2518.0,18983.0,43495.0
1998,9998,3538,3566,3653,4623,2973,2846,3629,2856,2931,...,728491.0,81701.0,53186.0,243349.0,88957.0,42461.0,525198.0,432330.0,54364.0,20355.0


In [43]:
patchByLaneTest = pd.read_csv('../data/patchByLaneTest.csv')
patchByLaneTest

,matchId,champWinPercent100_1,champWinPercent100_2,champWinPercent100_3,champWinPercent100_4,champWinPercent100_5,champWinPercent200_6,champWinPercent200_7,champWinPercent200_8,champWinPercent200_9,...,champScore100_1,champScore100_2,champScore100_3,champScore100_4,champScore100_5,champScore200_6,champScore200_7,champScore200_8,champScore200_9,champScore200_10
0,8000,0.5250,0.5138,0.5285,0.5168,0.5189,0.4919,0.5237,0.5008,0.4970,...,70.16,94.80,58.69,44.49,54.89,62.13,64.41,68.07,84.23,44.20
1,8001,0.4994,0.5138,0.4559,0.5333,0.5144,0.5173,0.4978,0.4949,0.5237,...,45.07,94.80,41.48,79.18,68.98,76.08,50.09,46.60,47.14,56.38
2,8002,0.4961,0.4837,0.5234,0.5222,0.5131,0.5193,0.5132,0.5145,0.4900,...,42.67,61.51,48.59,68.26,54.52,62.28,48.00,55.34,54.69,46.80
3,8003,0.5008,0.5013,0.5482,0.5221,0.5354,0.4912,0.4940,0.5387,0.4900,...,68.07,49.74,50.46,63.75,56.54,55.32,49.11,44.79,54.69,56.38
4,8004,0.5202,0.4753,0.4903,0.5295,0.5144,0.4883,0.5047,0.4956,0.5333,...,50.70,41.35,47.55,48.84,68.98,46.56,47.06,55.35,79.18,56.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,0.5070,0.4952,0.5072,0.4884,0.5136,0.4849,0.5190,0.5113,0.5241,...,60.06,55.79,58.89,66.87,62.45,49.47,53.91,50.11,47.33,66.08
1996,9996,0.4894,0.5138,0.4919,0.5210,0.4346,0.5173,0.5100,0.5035,0.5215,...,55.02,94.80,62.13,37.49,35.06,76.08,51.70,65.25,45.29,68.98
1997,9997,0.4801,0.5057,0.5035,0.5120,0.4915,0.5160,0.5080,0.4559,0.5333,...,59.11,49.29,65.25,58.64,54.17,90.92,58.21,41.48,79.18,96.39
1998,9998,0.4912,0.4795,0.5035,0.5167,0.5000,0.5160,0.4893,0.4919,0.5333,...,55.32,38.89,65.25,43.11,45.59,90.92,53.95,62.13,79.18,56.38


In [44]:
#adding patch test
results_p = pd.merge(results, patchByLaneTest, how = 'inner', on = 'matchId')
results_p

,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,champScore100_1,champScore100_2,champScore100_3,champScore100_4,champScore100_5,champScore200_6,champScore200_7,champScore200_8,champScore200_9,champScore200_10
0,8000,3063,4117,4365,3555,2213,3075,3884,3024,4150,...,70.16,94.80,58.69,44.49,54.89,62.13,64.41,68.07,84.23,44.20
1,8001,3784,3548,3203,4193,2764,3048,4045,2582,4168,...,45.07,94.80,41.48,79.18,68.98,76.08,50.09,46.60,47.14,56.38
2,8002,2681,4605,4309,2569,2042,3094,2827,3971,3504,...,42.67,61.51,48.59,68.26,54.52,62.28,48.00,55.34,54.69,46.80
3,8003,3471,3808,4186,4028,2291,4803,4072,2681,3130,...,68.07,49.74,50.46,63.75,56.54,55.32,49.11,44.79,54.69,56.38
4,8004,3181,3569,3693,3274,2624,3210,3997,3127,4044,...,50.70,41.35,47.55,48.84,68.98,46.56,47.06,55.35,79.18,56.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,3735,2545,4207,2930,2023,2992,3100,3282,3288,...,60.06,55.79,58.89,66.87,62.45,49.47,53.91,50.11,47.33,66.08
1996,9996,4162,4277,3313,4289,3082,3253,3173,4187,4079,...,55.02,94.80,62.13,37.49,35.06,76.08,51.70,65.25,45.29,68.98
1997,9997,2747,3392,2755,3269,2556,3728,3391,3492,4135,...,59.11,49.29,65.25,58.64,54.17,90.92,58.21,41.48,79.18,96.39
1998,9998,3538,3566,3653,4623,2973,2846,3629,2856,2931,...,55.32,38.89,65.25,43.11,45.59,90.92,53.95,62.13,79.18,56.38


In [66]:
dftest = results_p.dropna()

In [67]:
dftest.shape

(1998, 171)

In [70]:
testing_x = dftest.drop(['championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [45]:
testing_nas = results_p.drop(['championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [46]:
testing_nas.shape

(2000, 181)

In [47]:
testing_nas.columns[testing_nas.isna().any()].tolist()

['championLevel100_4',
 'championLevel200_7',
 'championPoints100_4',
 'championPoints200_7']

In [48]:

testing_nas = testing_nas.fillna(0)  

In [49]:
subm_y = gb_cv.predict(testing_nas)

In [50]:
subm_y

array([100, 200, 100, ..., 200, 100, 100], dtype=int64)

In [51]:
len(subm_y)

2000

In [52]:
submission = pd.read_csv('../data/sample_submission.csv')
submission

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,100
4,8004,100
...,...,...
1995,9995,100
1996,9996,100
1997,9997,100
1998,9998,100


In [53]:
submission['winner'] = subm_y
submission

,matchId,winner
0,8000,100
1,8001,200
2,8002,100
3,8003,100
4,8004,200
...,...,...
1995,9995,100
1996,9996,100
1997,9997,200
1998,9998,100


In [54]:
submission.to_csv('../data/submissionV14_3_31.csv', index = False)